# RAG QA Pipeline

This notebook demonstrates the complete Retrieval-Augmented Generation (RAG) pipeline for question answering. It loads the prepared data and embeddings, then executes queries to retrieve relevant context and generate answers.

## Setup Configuration

Initialize the configuration based on the execution environment (local using the HuggingFace API, local using an Ollama server or running in Colab using the HuggingFace API) and prepare the necessary directories.

The following parameters can be adjusted:
- `base_dir`: base directory where everything will be stored (it is recommended to use a mounted Google Drive directory if your are running in Colab, so the data can be stored persistently)
- `hf_cache_dir`: cache that will be used by the HuggingFace library
- `data_dir`: base directory where the dataset and embeddings will be stored (relative to `base_dir`)
- `train_dir`: directory path where the training split will be stored (relative to `data_dir`)
- `val_dir`: directory path where the validation split will be stored (relative to `data_dir`)
- `test_dir`: directory path where the test split will be stored (relative to `data_dir`)
- `embeddings_file`: file path where the pickle embeddings will be stored (relative to  `data_dir`; <span style="color:red;">deprecated</span>)
- `faiss_index_file`: file path where the faiss embeddings will be stored (relative to  `data_dir`)
- `passages_file`: file path where the pickle file containing the passages will be stored (relative to  `data_dir`)
- `embedding_model`: name of the embedding model to use
- `rerank_model`: name of the reranker model to use
- `generator_model`: name of the generator model to use (in case of an Ollama model, make sure it is installed)
- `val_split_size`: size of the validation split (default is 7900, as specified in the assignment description)
- `shard_batch_size`: number of samples that each shard contains (can be adjusted, depending on the available RAM)
- `chunk_tokens`: 
- `chunk_overlap`: 
- `embeddings_batch_size`: 

In [4]:
from src.config import OllamaConfig, LocalConfig, ColabConfig, is_colab

USE_OLLAMA = True

if USE_OLLAMA:
    OLLAMA_HOST = "172.19.176.1"
    OLLAMA_PORT = 11434
    OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}/api/chat"
    config = OllamaConfig(ollama_url=OLLAMA_URL)
else:
    config = ColabConfig() if is_colab() else LocalConfig()
    
config.ensure_dirs()

✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/.hf_cache
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data/train
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data/validation
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data/test


## Load Embeddings

Load the precomputed corpus and embeddings from the data preparation step.

In [2]:
from src.load_data import load_embeddings

corpus, emb = load_embeddings(config=config)

/home/lucas/.virtualenvs/rag-qa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔹 Loaded FAISS index with 978526 passages


## Query and Generate Answer

Execute a sample query through the RAG pipeline. The retriever fetches relevant context passages, and the generator produces an answer based on that context.

In [8]:
# initialize the retriever in an own cell, so you can
from src.retriever import  Retriever

retriever = Retriever()

In [10]:
from src.generator import generate_answer_combined

query = "Who invented the speed of light?"
answer, ctx = generate_answer_combined(query, retriever, corpus, emb, config=config, top_k=5)

print("\n🔍 Used Context Passages:\n")
for i,p in enumerate(ctx,1):
    print(f"{i}. {p[:200].replace(chr(10),' ')}...\n")

print("💡 Final Answer:\n", answer)


🔍 Used Context Passages:

1. Speed of light: snell's law using the opposing assumption, the more dense the medium the slower light traveled. fermat also argued in support of a finite speed of light. first measurement attempts in ...

2. Speed of light: or eight minutes " for the time taken for light to travel from the sun to the earth ( the modern value is 8 minutes 19 seconds ). newton queried whether rømer's eclipse shadows were co...

3. Light: the speed of light throughout history. galileo attempted to measure the speed of light in the seventeenth century. an early experiment to measure the speed of light was conducted by ole rømer, ...

4. Speed of light: a value of in 1862. in the year 1856, wilhelm eduard weber and rudolf kohlrausch measured the ratio of the electromagnetic and electrostatic units of charge, 1 / √ε0μ0, by discharging ...

5. Speed of light: ##r bodies. by the 14th century, sayana had made statements about the speed of light in his commentary on the hindu rigved